In [1]:
import sys
sys.path.append('/scratch/anaconda3/lib/python3.6/site-packages')

In [69]:
import spacy
import torch
from torchtext.data import Field, BucketIterator, TabularDataset
import torchtext.data as data
from gensim.utils import tokenize
import pandas as pd

In [ ]:
#just some debugging stuff to check tmx
tree = parse('./cs-pl.tmx')
root = tree.getroot()[1] #This gets the body of the tmx file
print(len(root))
print(root[0][0][0].text)

l1 = list(root[0][0].attrib.keys())[0]
l1 = root[0][0].attrib[l1]
print(l1)
print(root[0][1][0].text)
l2 = list(root[0][1].attrib.keys())[0]
l2 = root[0][1].attrib[l2]
print(l2)

root.find('tuv')

In [192]:
from xml.etree.ElementTree import parse
def TMXtoTxt(path, L1, L2):
    tree = parse(path) #you can parse TMX as an xml file
    root = tree.getroot()[1] #should get body of tmx (where pairs are)
    
    data = {L1: [None] * len(root), L2: [None] * len(root)}
    kept = 0
    for i , child in enumerate(root):
        l1 = list(child[0].attrib.keys())[0]
        l1 = child[0].attrib[l1]
        l2 = list(child[1].attrib.keys())[0]
        l2 = child[1].attrib[l2]
        
        assert l1 == L1 and l2 == L2, "seems there is a misalignment of language {} != {} or {} != {}".format(l1, L1, l2, L2)
        if len(child[0][0].text) > 0 and len(child[1][0].text) > 0:
            data[l1][i] = child[0][0].text
            data[l2][i] = child[1][0].text
            kept += 1
    print("number kept {} / {}".format(kept, i))
    
    assert len(data[l1]) == len(data[l2]), "You don't have equal pairs"
    #something like this
    return data    

def checkTSV(path):
    with open(path, encoding='utf-8') as f:
        i = 0
        for l in f:
            pair = l.strip().split('\t')
            if len(pair) > 2 or len(pair) == 1:
                print('\n')
                print(l.strip())
                print(i, pair)
            i += 1
    print(i)
    
def recoverEntry(entries):
    #here are the 2 patterns i've seen europarl (do not use this as a general solution)
    if len(entries) == 3 and len(entries[1]) == 0: #this means we had 2 entries in between each entry
        return [entries[0], entries[-1]]
    elif len(entries) == 3 and (entries[0].strip() in entries[2]): #i have seen this...once, very skeptical if this will work
        return [entries[0] + entries[1], entries[2]]
    elif len(entries) == 3 and (entries[0].strip() not in entries[2]):
        return [entries[1], entries[2]] #again 1 bug that happened in es-pt
    elif len(entries) == 4 and entries[0] == entries[2]: #Weird split from numbers
        return [entries[0] + entries[1], entries[2] + entries[3]]
    else:
        print('failed to recover:')
        print(entries)
        return []
    
def cleanTSV(path, l1, l2, thresh=3):
    results = {l1:[], l2: []}
    with open(path, encoding='utf-8') as f:
        i = 0
        for l in f:
            i += 1
            entries = l.strip().split('\t')
            if len(entries) > 2:
                entries = recoverEntry(entries)
            if len(entries) <= 1:
                continue #there's no pair so just ignore it
            non_empty = len(entries[0]) > 0 and len(entries[1]) > 0
            long_enough = len(entries[0].strip()) > thresh and len(entries[0].strip()) > thresh 
            if len(entries) == 2 and non_empty and long_enough :
                results[l1].append(entries[0])
                results[l2].append(entries[1])
    assert len(results[l1]) == len(results[l2]), "You don't have an equal number of sentences"
    print('Entries kept {} / {}'.format(len(results[l1]), i))
    return results

def merge_all_data(dicts, l1, l2):
    all_data = {l1: [], l2: []}
    for d in dicts:
        all_data[l1] =  all_data[l1] + d[l1].copy()
        all_data[l2] =  all_data[l2] + d[l2].copy()
        assert len(all_data[l1]) == len(all_data[l2]), "ugh oh, unaligned bitext"
    return all_data       

def lookup_words(x, vocab=None):
    if vocab is not None:
        x = [vocab.itos[i] for i in x]
    return [str(t) for t in x]

def buildVocab(sentences, tokenizer,min_freq=1):
    vocab = {}
    total_tokens = 0
    for sent in sentences:
        for t in tokenizer(sent):
            total_tokens += 1
            if t in vocab:
                vocab[t] += 1
            else:
                vocab[t] = 1

    vocab = [k for k in vocab.keys() if vocab[k] > min_freq]
    print('Vocab size {} with min_freq {}'.format(len(vocab), min_freq))
    print('Total tokens counted {}'.format(total_tokens))
    return vocab
        

In [100]:
#Czech - polish translation data

#checkTSV('./europarl-v9.cs-pl.tsv')
#checkTSV('./wikititles-v1.cs-pl.tsv')
pth='./cs-pl/'
#Handle all of the datasets 
JRC_cs_pl = TMXtoTxt(pth + 'cs-pl.tmx', 'cs', 'pl')
europarl_es_pt = cleanTSV(pth + 'europarl-v9.cs-pl.tsv', 'cs', 'pl')
wikititles_es_pt = cleanTSV(pth + 'wikititles-v1.cs-pl.tsv', 'cs', 'pl')


pd.DataFrame.from_dict(JRC_cs_pl).to_csv(pth + 'clean-jrc-cs-pl.tsv',sep='\t',index=False)
pd.DataFrame.from_dict(europarl_cs_pl).to_csv(pth + 'clean-europarl-cs-pl.tsv',sep='\t',index=False)
pd.DataFrame.from_dict(wikititles_cs_pl).to_csv(pth + 'clean-wikititles-cs-pl.tsv',sep='\t',index=False)

#Merge all the data together
cs_pl = [JRC_cs_pl, europarl_cs_pl, wikititles_cs_pl]
all_cs_pl = merge_all_data(cs_pl, 'cs', 'pl')    
pd.DataFrame.from_dict(all_cs_pl).to_csv(pth + 'clean-all-cs-pl.tsv',sep='\t',index=False)

#Handle dev set 
cs_pl_dev = {'cs':[], 'pl': []}
with open('pth + ted-dev-cspl/dev.cs', encoding='utf-8') as f:
    cs_pl_dev['cs'] = list(f.read().split('\n'))
with open('pth + ted-dev-cspl/dev.pl', encoding='utf-8') as f:
    cs_pl_dev['pl'] = list(f.read().split('\n'))
pd.DataFrame.from_dict(cs_pl_dev).to_csv(pth + 'clean-dev-cs-pl.tsv',sep='\t',index=False)

In [ ]:
ml = spacy.load('xx_ent_wiki_sm')
def tokenize_ml(sentence):
    return [tok.text for tok in ml.tokenizer(sentence)]

def simple_tokenize(sentence):
    return [tok for tok in tokenize(sentence)]

CS_TEXT = Field(tokenize=tokenize_ml)
PL_TEXT = Field(tokenize=tokenize_ml)

data_fields = [('cs', CS_TEXT), ('pl', PL_TEXT)] 
train,val = TabularDataset.splits(path='./', train='clean-all-cs-pl.tsv', validation='clean-dev-cs-pl.tsv',
                                  format='tsv', fields=data_fields)

CS_TEXT.build_vocab(train, val, min_freq=5)
PL_TEXT.build_vocab(train, val, min_freq=5)

In [ ]:
bucket_iter = BucketIterator(train, batch_size=20, \
sort_key=lambda x: len(x.cs), shuffle=True)
batch = next(iter(bucket_iter))
#print(batch.cs)
#print(batch.pl)


In [ ]:
#Just something to keep in mind when translating across the dimensions
CS = [" ".join(lookup_words(x, CS_TEXT.vocab)) for x in batch.cs.t()]
PL = [ " ".join(lookup_words(x, PL_TEXT.vocab)) for x in batch.pl.t()]
for pair in zip(CS,PL):
    print('-'* 115)
    print(pair[0])
    print(pair[1])

In [187]:
pth = './es-pt/'
checkTSV(pth + 'europarl-v9.es-pt.tsv')
#checkTSV(pth + 'wikititles-v1.es-pt.tsv')
#checkTSV(pth +  'news-commentary-v14.es-pt.tsv')




810 ['']


.
1657 ['.']


Quiero completar aún algo ya que usted se ha referido a ello.
2994 ['Quiero completar aún algo ya que usted se ha referido a ello.']


Muy especialmente, su ponente subraya algunas propuestas, principalmente el mantenimiento del statu quo sobre las 6/12 millas.
3497 ['Muy especialmente, su ponente subraya algunas propuestas, principalmente el mantenimiento del statu quo sobre las 6/12 millas.']


Igualmente para el mantenimiento de zonas de veda después de las comprobaciones científicas, forzosamente con la cuestión del acceso al mar del Norte que va a ser más vigoroso, habida cuenta de las ampliaciones.
3498 ['Igualmente para el mantenimiento de zonas de veda después de las comprobaciones científicas, forzosamente con la cuestión del acceso al mar del Norte que va a ser más vigoroso, habida cuenta de las ampliaciones.']



4001 ['']


.
4759 ['.']


.
6545 ['.']


Por tanto, ha llegado el momento de domesticar al automóvil.
7342 ['Por tanto, ha llegado el 

68782 ['']


Para terminar, não posso deixar de felicitar os dois relatores e, em especial, pelo que me diz respeito, em nome da Comissão das Pescas, o senhor deputado Bernard Poignant.
68917 ['Para terminar, não posso deixar de felicitar os dois relatores e, em especial, pelo que me diz respeito, em nome da Comissão das Pescas, o senhor deputado Bernard Poignant.']


2.
69173 ['2.']


Porém, em simultâneo, todas estas informações deverão ter um tratamento confidencial.
69290 ['Porém, em simultâneo, todas estas informações deverão ter um tratamento confidencial.']


Debemos evitar esto.
69569 ['Debemos evitar esto.']



70618 ['']


Eles estão à espera do nosso sim e espero que venham a tê­lo amanhã.
70781 ['Eles estão à espera do nosso sim e espero que venham a tê\xadlo amanhã.']


relator.
70861 ['relator.']


Además de lograr, como es lógico, una reducción en las emisiones de gases de efecto invernadero.
71371 ['Además de lograr, como es lógico, una reducción en las emisiones de gas




137238 ['']


El resultado de la conciliación sobre el transporte por carretera debe ser muy bien recibido.
137947 ['El resultado de la conciliación sobre el transporte por carretera debe ser muy bien recibido.']


Hubiera sido imposible conseguir que el Consejo lo aprobara.
137961 ['Hubiera sido imposible conseguir que el Consejo lo aprobara.']



138330 ['']



141733 ['']


.
142812 ['.']


A UE considera ainda ser indispensável elaborar e implementar com urgência, e independentemente das contingências do processo de paz, um programa de profundas reformas socioeconómicas decisivas.
143865 ['A UE considera ainda ser indispensável elaborar e implementar com urgência, e independentemente das contingências do processo de paz, um programa de profundas reformas socioeconómicas decisivas.']


.
144135 ['.']


Tenemos que trabajar en ello.
144246 ['Tenemos que trabajar en ello.']


En mi intervención me voy a dedicar al futuro de la política regional.
144333 ['En mi intervención me voy a



La Comisión se centrará ahora en la preparación de medidas de ejecución.
206366 ['La Comisión se centrará ahora en la preparación de medidas de ejecución.']


Ésta es, sin duda, una tarea particularmente difícil y estoy seguro de que el Canciller Schröder se ha percatado de ello y seguirá percatándose en el futuro.
206369 ['Ésta es, sin duda, una tarea particularmente difícil y estoy seguro de que el Canciller Schröder se ha percatado de ello y seguirá percatándose en el futuro.']


En cuanto a la situación del sector en Austria y Alemania, prefiero no hacer ninguna observación y ceñirme al Reino Unido.
206399 ['En cuanto a la situación del sector en Austria y Alemania, prefiero no hacer ninguna observación y ceñirme al Reino Unido.']



206479 ['']


É mais a forma do texto que é criticável: trata-se de um regulamento, directamente aplicável em todos os países membros.
207553 ['É mais a forma do texto que é criticável: trata-se de um regulamento, directamente aplicável em todos os p



Como vários oradores já sublinharam, e desde logo o Presidente Hernández Mollar, as nossas companhias aéreas estão num dilema.
262997 ['Como vários oradores já sublinharam, e desde logo o Presidente Hernández Mollar, as nossas companhias aéreas estão num dilema.']



263908 ['']


Há quatro aspectos que gostaria de sublinhar.
264097 ['Há quatro aspectos que gostaria de sublinhar.']


Al contrario, deberán reembolsar la ayuda que hayan recibido en el primer país.
264766 ['Al contrario, deberán reembolsar la ayuda que hayan recibido en el primer país.']



265276 ['']



265954 ['']


.
265957 ['.']


A União Europeia não foi capaz de demonstrar a necessária unidade.
266305 ['A União Europeia não foi capaz de demonstrar a necessária unidade.']


Não pôde, pois, lamentavelmente, exercer a sua influência como teríamos desejado.
266306 ['Não pôde, pois, lamentavelmente, exercer a sua influência como teríamos desejado.']


Espero que agora sejamos capazes de, pelo menos, primeiramente recu

...

Em primeiro lugar, a questão da independência da UCLAF.
1529269 ['Em primeiro lugar, a questão da independência da UCLAF.']


Concordo totalmente que a UCLAF deve trabalhar de forma independente.
1529271 ['Concordo totalmente que a UCLAF deve trabalhar de forma independente.']


O mesmo se aplica ao seu trabalho de acompanhamento das investigações.
1529272 ['O mesmo se aplica ao seu trabalho de acompanhamento das investigações.']



1529821 ['']


, por escrito.
1530545 [', por escrito.']



1532558 ['']


É isso que gostaria de transmitir ao senhor presidente Gil-Robles.
1532585 ['É isso que gostaria de transmitir ao senhor presidente Gil-Robles.']


La propuesta de insistir en el 'oro nórdico' en las nuevas monedas de euros es un disparate oneroso».
1532601 ["La propuesta de insistir en el 'oro nórdico' en las nuevas monedas de euros es un disparate oneroso»."]


Un primer grupo de enmiendas es relativo al abastecimiento de cacao.
1532826 ['Un primer grupo de enmiendas es relativo al




1592169 ['']


A explosão do sector que se prevê para os próximos anos vai acarretar uma multiplicação dos problemas.
1592546 ['A explosão do sector que se prevê para os próximos anos vai acarretar uma multiplicação dos problemas.']


Isso exige que a União intervenha em diversos aspectos.
1592547 ['Isso exige que a União intervenha em diversos aspectos.']


Para avaliar o cumprimento das directivas e regulamentos existentes.
1592548 ['Para avaliar o cumprimento das directivas e regulamentos existentes.']


Para articular as medidas no domínio do turismo com as da protecção do consumidor, promovendo a cooperação entre a DG XXIII e a DG XXIV.
1592549 ['Para articular as medidas no domínio do turismo com as da protecção do consumidor, promovendo a cooperação entre a DG XXIII e a DG XXIV.']


Para garantir o acesso rápido de turistas e de profissionais do turismo à justiça e criar formas de arbitragem para conflitos intra-comunitários.
1592550 ['Para garantir o acesso rápido de turista




1656708 ['']



1657731 ['']


La Comisión Europea está al corriente de la cuestión relativa a las Islas Chagos pero nunca ha participado en las discusiones oficiales sobre el particular.
1659944 ['La Comisión Europea está al corriente de la cuestión relativa a las Islas Chagos pero nunca ha participado en las discusiones oficiales sobre el particular.']


Estamos tratando aquí de varias cuestiones diferentes.
1660004 ['Estamos tratando aquí de varias cuestiones diferentes.']



1660946 ['']


Foi, aliás, por este motivo que propus a criação de uma agência europeia do turismo.
1661920 ['Foi, aliás, por este motivo que propus a criação de uma agência europeia do turismo.']


Confío en que esto sea el inicio de un debate que concluya positivamente para toda Europa.
1662542 ['Confío en que esto sea el inicio de un debate que concluya positivamente para toda Europa.']


1, 6, 7, 9, 11, 15, 17 y 19.
1663509 ['1, 6, 7, 9, 11, 15, 17 y 19.']



1663993 ['']


B4-0902/98 del Sr. Collins, en



Lo reconozco.
1727021 ['Lo reconozco.']



1727039 ['']



1727860 ['']



1728963 ['']


una primera parte, que se refiere a las disposiciones de aplicación de los procedimientos reglamentarios y de las disposiciones que la Asamblea aprueba por mayoría simple; -una segunda categoría, que se refiere a las disposiciones adoptadas con arreglo a normas específicas que figuran en el propio Reglamento y en particular, por ejemplo -se da este ejemplo como explicación-, el artículo 9, relativo a las normas de conducta.
1729210 ['una primera parte, que se refiere a las disposiciones de aplicación de los procedimientos reglamentarios y de las disposiciones que la Asamblea aprueba por mayoría simple; -una segunda categoría, que se refiere a las disposiciones adoptadas con arreglo a normas específicas que figuran en el propio Reglamento y en particular, por ejemplo -se da este ejemplo como explicación-, el artículo 9, relativo a las normas de conducta.']


El procedimiento es siempre el mismo.




E aqui voltamos a estar em desacordo com o que é ou não de direito na União Europeia, e esse é um cenário no qual não estamos interessados.
1793183 ['E aqui voltamos a estar em desacordo com o que é ou não de direito na União Europeia, e esse é um cenário no qual não estamos interessados.']


, Comisión.
1793653 [', Comisión.']


Pero también porque la Comisión y yo mismo y mi servicio en particular hemos estado examinando la cuestión de la creación de una estructura apropiada para una agencia sobre seguridad alimentaria.
1793655 ['Pero también porque la Comisión y yo mismo y mi servicio en particular hemos estado examinando la cuestión de la creación de una estructura apropiada para una agencia sobre seguridad alimentaria.']



1794335 ['']


¡Esto también hay que decirlo!
1795466 ['¡Esto también hay que decirlo!']


Na era da globalização, não podemos virar as costas às potencialidades de desenvolvimento oferecidas pelo comércio livre e pela concorrência.
1796092 ['Na era da global

In [193]:
#Spanih Portugese translation
pth = './es-pt/'
#Clean up each of the datasets
JRC_es_pt = TMXtoTxt(pth +'es-pt.tmx', 'es', 'pt')
europarl_es_pt = cleanTSV(pth + 'europarl-v9.es-pt.tsv', 'es', 'pt')
wikititles_es_pt = cleanTSV(pth + 'wikititles-v1.es-pt.tsv', 'es', 'pt')
news_commentary = cleanTSV(pth + 'news-commentary-v14.es-pt.tsv', 'es','pt')

#Create the TSV files of each 
pd.DataFrame.from_dict(JRC_es_pt).to_csv(pth + 'clean-jrc-es-pt.tsv',sep='\t',index=False)
pd.DataFrame.from_dict(europarl_es_pt).to_csv(pth + 'clean-europarl-es-pt.tsv',sep='\t',index=False)
pd.DataFrame.from_dict(wikititles_es_pt).to_csv(pth + 'clean-wikititles-es-pt.tsv',sep='\t',index=False)
pd.DataFrame.from_dict(news_commentary).to_csv(pth + 'clean-newcommentary-es-pt.tsv',sep='\t',index=False)

# merge all the train data
cs_pl = [JRC_es_pt, europarl_es_pt, wikititles_es_pt, news_commentary]
all_cs_pl = merge_all_data(cs_pl, 'es', 'pt')    
pd.DataFrame.from_dict(all_cs_pl).to_csv(pth + 'clean-all-es-pt.tsv',sep='\t',index=False)

#convert the dev sets to tsv
cs_pl_dev = {'es':[], 'pt': []}
with open(pth + 'at-dev-espt/dev.es', encoding='utf-8') as f:
    cs_pl_dev['es'] = list(f.read().split('\n'))
with open(pth + 'at-dev-espt/dev.pt', encoding='utf-8') as f:
    cs_pl_dev['pt'] = list(f.read().split('\n'))
pd.DataFrame.from_dict(cs_pl_dev).to_csv(pth + 'clean-dev-es-pt.tsv',sep='\t',index=False)

number kept 1067198 / 1067197
failed to recover:
['13. ', 'Acuerdo de Asociación UE-Centroamérica (votacion)', 'Acordo de Associação UE-América Central *** (votação)']
Entries kept 1791082 / 1811977
Entries kept 614600 / 621296
Entries kept 46850 / 48168


In [200]:
#Create the Hindi Nepali dataset
#slightly different from all the other ones but not by much 
def getDataFromFile(pth, name, l1, l2):
    l1_l2_dict = {l1:[], l2: []}
    with open(pth + name + '.' + l1, encoding='utf-8') as f:
        l1_l2_dict[l1] = list(f.read().split('\n'))
    with open(pth + name + '.' + l2, encoding='utf-8') as f:
        l1_l2_dict[l2] = list(f.read().split('\n'))
    assert len(l1_l2_dict[l1]) == len(l1_l2_dict[l2]), "ugh oh, unaligned"
    print(len(l1_l2_dict[l1]))
    return l1_l2_dict
pth = './hi-ne/TrainDevSimilar/'

#train set
he_ne_train = getDataFromFile(pth, 'train', 'hi', 'ne')
pd.DataFrame.from_dict(he_ne_train).to_csv(pth + 'clean-train-hi-ne.tsv',sep='\t',index=False)
#dev set 
he_ne_dev = getDataFromFile(pth, 'dev2019', 'hi', 'ne')
pd.DataFrame.from_dict(he_ne_dev).to_csv(pth + 'clean-dev-hi-ne.tsv',sep='\t',index=False)

65506
3001


In [3]:
#en = spacy.load('en')
#fr = spacy.load('fr')

#def tokenize_en(sentence):
#    return [tok.text for tok in en.tokenizer(sentence)]

#def tokenize_fr(sentence):
#    return [tok.text for tok in fr.tokenizer(sentence)]

#EN_TEXT = Field(tokenize=tokenize_en)
#FR_TEXT = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [24]:
import pandas as pd

raw_data = {'English' : [line for line in europarl_en], 'French': [line for line in europarl_fr]}

df = pd.DataFrame(raw_data, columns=["English", "French"])

# remove very long sentences and sentences where translations are 
# not of roughly equal length

df['eng_len'] = df['English'].str.count(' ')
df['fr_len'] = df['French'].str.count(' ')
df = df.query('fr_len < 10 & eng_len < 10')
df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')
df['fr_len'][0:10]

In [26]:
from sklearn.model_selection import train_test_split

# create train and validation set 
train, val = train_test_split(df, test_size=0.1)

train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)

In [27]:
# associate the text in the 'English' column with the EN_TEXT field, # and 'French' with FR_TEXT
data_fields = [('English', EN_TEXT), ('French', FR_TEXT)]

train,val = TabularDataset.splits(path='./', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

In [33]:
FR_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)
bucket_iter = BucketIterator(train, batch_size=20, \
sort_key=lambda x: len(x.French), shuffle=True)

In [34]:
import time
#batch = next(iter(train_iter))
times = []
s = time.time()
for b in bucket_iter:
    n = time.time()
    times.append(n - s)
    s = n
print(sum(times) / len(times))

0.000519905676041515


In [30]:
# code from http://nlp.seas.harvard.edu/2018/04/03/attention.html 
# read text after for description of what it does

global max_src_in_batch, max_tgt_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.English))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.French) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [35]:
train_iter = MyIterator(train, batch_size=1300, device=0,
                        repeat=False, sort_key= lambda x:
                        (len(x.English), len(x.French)),
                        batch_size_fn=batch_size_fn, train=True,
                        shuffle=True)

times = []
s = time.time()
for b in train_iter:
    n = time.time()
    times.append(n - s)
    s = n
print(sum(times) / len(times))


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


0.0029750237237150872
